# Metaprogramming

Julia, igual que Lisp, representa al código como una estructura de datos en el propio lenguaje. La posibilidad de escribir código que genere y modifique código es lo que se entiende por "Metaprogramming".

Ilustraremos algunos conceptos siguiendo el [manual](http://docs.julialang.org/en/release-0.4/manual/metaprogramming/). 


Empecemos por decir lo que en algún sentido suena medio trivial, que es que cualquier línea de código, inicialmente se considera como una cadena:

In [ ]:
one_plus_one = "1+1"

Lo que hace Julia internamente es convertir esa cadena en una *expresión*:

In [ ]:
ex1 = parse(one_plus_one)

In [ ]:
typeof(ex1)

In [ ]:
?Expr

Claramente, un objeto tipo `Expr` tiene tres partes:

- Un `Symbol`, que se guarda en `head`:

In [ ]:
ex1.head

- Un vector (tipo `Any`) que contiene a los argumentos:

In [ ]:
ex1.args

- el tipo resultante de la expresión, que puede ser anotado por el usuario

In [ ]:
ex1.typ

Las expresiones también pueden ser escritas directamente, respetando el órden de los campos que definen la expresión:

In [ ]:
ex2 = Expr(:call, :+, 1, 1)

El punto importante aquí es que el código en Julia está representado internamente por expresiones escritas en Julia.

La función `dump()` da información sobre la expresión:

In [ ]:
dump(ex2)

Expresiones más complicadas pueden ser construidas:

In [ ]:
ex3 = parse("(4 + 4) / 2")

In [ ]:
dump(ex3)

In [ ]:
Meta.show_sexpr(ex3)

Uno de los usos de `:` es crear símbolos; uno también puede usar `symbol()`:

In [ ]:
:foo == symbol("foo")

In [ ]:
symbol(:var,'_',"sym")

Otro uso de `:` es crear expresiones sin usar `Expr`; a esto se le llama *quoting*.

In [ ]:
ex = :(a+b*c+1)

In [ ]:
dump(ex)

Lo interesante, es que uno puede cambiar *programáticamente* el contenido de una expresión. Por ejemplo, en lugar de $ex=a + b*c + 1$ quiero tener $ex=a + b*c + 2.1$:

In [ ]:
Meta.show_sexpr(ex)

Dado que los argumentos de una expresión están en Julia, uno puede modificarlos. Por ejemplo:

In [ ]:
ex.args[4] = 2.1

In [ ]:
Meta.show_sexpr(ex)

Otra alternativa para definir expresiones complejas es usando el bloque `quote ... end`

In [ ]:
# Lo siguiente es equivalente a:
# ex = :(x=1; y=2; x+y)
ex = quote
    x = 1
    y = 2
    x + y
end

In [ ]:
Meta.show_sexpr(ex)

Para evaluar una expresión (de tipo `Expr`), usamos `eval()`. Respecto a esto, vale la pena notar que, mientras no se ha evaluado una expresión las variables internas no tienen ningún valor; sin embargo, una vez evaluada, los valores de las variables internas quedan definidas. Esto puede causar problemas, llamados *side effects*.

In [ ]:
x

In [ ]:
y = 3

In [ ]:
eval(ex)

In [ ]:
x

In [ ]:
y # y queda sobreescrita al evaluar la expresión !!

Uno incluso puede definir el valor de ciertas variables y usarlo como tal, o sustituirlo directamente:

In [ ]:
z = 4

In [ ]:
:(2*x+z)

In [ ]:
eval(ans)

In [ ]:
:(2*x+$z)

In [ ]:
:(2*x+$zz)

El punto es que uno puede definir funciones, que regresen expresiones, que pueden ser evaluadas

Un ejemplo más interesante es implementar los polinomios de Wilkinson, que se definen como:

\begin{equation}
    W_n(x) = \prod_{i=1}^n (x-i) = (x-1) (x-2) \cdots (x-n).
\end{equation}

Ciertamente este ejemplo se puede implementar a través de una recursión; implementémoslo generando código escrito por Julia.

In [ ]:
nombre(n::Int) = symbol( string("W_", n) )

In [ ]:
nombre(3)

In [ ]:
function wilkinson(n::Int)
    # Checo que `n` sea >= 1
    @assert n ≥ 1   # ≥ se obtiene con \ge<TAB>
    
    ex = :(x-1)
    for i = 2:n
        ex = :( ($ex) * ( x-$i) )
    end
    ex_ret = :( $(nombre(n))(x) = $ex )
    ex_ret
end

In [ ]:
wilkinson(0)

In [ ]:
wilkinson(3)

In [ ]:
eval(ans)

In [ ]:
W_3(2)

In [ ]:
eval(wilkinson(4))

In [ ]:
W_4(2.5)

**Ejercicio 1:** Implementen el factorial con metaprogramming.

## Macros

En varias ocasiones hemos usado ya algunas instrucciones que incluyen `@` antes, por ejemplo, `@time` o `@assert`. Este tipo de funciones son *macros*: Los macros son funciones cuyas entradas son *expresiones*, que en algún sentido son manipuladas y al final evaluadas.

In [ ]:
macro simple_example(expr)
    @show expr   # this is another macro !
    return 0     # for simplicity
end

In [ ]:
@simple_example(x+y)

In [ ]:
@simple_example a = x + y

Cambiemos un poco el macro `@simple_example`; esta versión devuelve la expresión que es evaluada:

In [ ]:
macro simple_example(expr)
    @show expr   # this is another macro !
    expr         # returns the input expression
end

In [ ]:
x, y

In [ ]:
@simple_example x + y

In [ ]:
x = 3; y = 7;
@simple_example x + y

La gran sutileza de los macros, a diferencia de las funciones que hasta ahora hemos visto, es que permiten introducir y modificar código *antes* de que sea ejecutado, dado que los macros son ejecutados cuando el código es leído (*parse time*).

El siguiente ejemplo está tomado del manual.

In [ ]:
macro twostep(arg)
    println("I execute at parse time. The argument is: ", arg)

    return :(println("I execute at runtime. The argument is: ", $arg))
end

In [ ]:
ex = macroexpand( :(@twostep :(1, 2, 3)) );

In [ ]:
macroexpand( (:@simple_example x+y) )

In [ ]:
typeof(ex)

In [ ]:
ex

In [ ]:
eval(ex)

In [ ]:
@twostep :(1, 2, 3)

## Generación de código

Arriba, con la definición `wilkinson()` de hecho definimos una función que genera código. Si bien ése no es el mejor ejemplo para esto, el punto es que a veces, excepto por pequeños cambios (por ejemplo, en los operadores), el código esencialmente se repite.


Como ejemplo, usaremos el tipo `AuDiff`, y sus métodos:

In [ ]:
type AuDiff{T<:Real}
    fun :: T
    der :: T
end

AuDiff(a, b) = AuDiff(promote(a,b)...)

AuDiff{T<:Real}(a::T) = AuDiff(a, zero(T))

In [ ]:
import Base: convert

convert(::Type{AuDiff}, b::Real) = AuDiff(b)
convert{T<:Real, S<:Real}(::Type{AuDiff{T}}, b::S) = AuDiff(convert(T,b))
convert{T<:Real}(::Type{AuDiff{T}}, b::T) = AuDiff(b)
convert{T<:Real, S<:Real}(::Type{AuDiff{T}}, b::AuDiff{S}) = 
    AuDiff(convert(T,b.fun),convert(T,b.der))


In [ ]:
import Base: promote_rule, promote

promote_rule{T<:Real}(::Type{AuDiff{T}}, ::Type{T}) = AuDiff{T}
promote_rule{T<:Real, S<:Real}(::Type{AuDiff{T}}, ::Type{S}) = AuDiff{promote_type(T, S)}

In [ ]:
import Base: +, -, *, /, ^

# Aquí definimos `+` y `-`
for op = (:+, :-)
    @eval ($op)(a::AuDiff, b::AuDiff) = AuDiff( ($op)(a.fun,b.fun), ($op)(a.der,b.der))
end

*(a::AuDiff, b::AuDiff) = AuDiff(a.fun*b.fun, a.fun*b.der+a.der*b.fun)

function /(a::AuDiff, b::AuDiff)
    nn = a.fun/b.fun
    dd = (a.der - nn*b.der)/b.fun
    return AuDiff(nn, dd)
end

^(a::AuDiff, n::Int) = AuDiff(a.fun^n, n * a.der * a.fun^(n-1))

In [ ]:
sin(AuDiff(0.0,1.0))

In [ ]:
import Base: exp, log, sin, cos, tan

tup_fn   = [:exp, :log, :sin, :cos, :tan]
tup_fder = [:exp, :(x -> 1/x), :(x -> cos(x)), :(x -> -sin(x)), :(x -> (sec(x))^2)]

for (fn, fnder) in zip(tup_fn, tup_fder)
    @eval ($fn)(a::AuDiff) = AuDiff( ($fn)(a.fun), a.der*($fnder)(a.fun) )
end

In [ ]:
sin(AuDiff(0.0,1.0))